In [9]:
import os
import sys
import xml.etree.ElementTree as ElementTree
import collections
from collections import Counter
import pickle
import json

import psycopg2

import parse_texte

In [2]:
dossier_data = '/home/michel/loi/data/'


## Charge les infos

In [3]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

curseur.execute("select base_origine, categorie, cid from texte where valide = True;")
liste_texte = curseur.fetchall()

curseur.close()
connection.close()

In [4]:
liste_id_par_cid = {}
valeurs_struct_par_id_par_cid = {}
valeurs_version_par_id_par_cid = {}

i = 0
for base_origine, categorie, cid in liste_texte:
    nom_fichier = dossier_data + 'parse_texte/' + cid

    with open(nom_fichier, 'rb') as f:
        liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = pickle.load(f)
        
        liste_id_par_cid[cid] = liste_id_
        valeurs_struct_par_id_par_cid[cid] = valeurs_struct_par_id_
        valeurs_version_par_id_par_cid[cid] = valeurs_version_par_id_
        
        i += 1
        if i % 100000 == 0:
            print(i)


100000
200000
300000
400000
500000
600000
700000
800000
900000


## Traitements

Création de la nouvelle table contenant les versions :
```
create table texte_version (
    base_origine varchar(21),
    categorie varchar(21),
    cid varchar(21),
    id_ varchar(21),
    valide boolean,
    
    ID_ELI text,
    ID_ELI_ALIAS text,
    ANCIEN_ID text,
    ORIGINE text,
    URL_version text,
    URL_struct text,
    NATURE text,
    NUM text,
    NUM_SEQUENCE text,
    NOR text,
    DATE_PUBLI text,
    DATE_TEXTE text,
    DERNIERE_MODIFICATION text,
    VERSIONS_A_VENIR text,
    ORIGINE_PUBLI text,
    PAGE_DEB_PUBLI text,
    PAGE_FIN_PUBLI text,
    
    TITRE text,
    TITREFULL text,
    ETAT text,
    DATE_DEBUT text,
    DATE_FIN text,
    AUTORITE text,
    MINISTERE text,
    MCS_TXT text,
    LIENS text,
    NOTICE text,
    VISAS text,
    SIGNATAIRES text,
    TP text,
    NOTA text,
    ABRO text,
    RECT text,
    SM text,
    
    VERSIONS text,
    LIENS_ART text,
    LIENS_SECTION_TA text
);
create index on texte_version (id_);
```

In [55]:
def traite_cid(base_origine, categorie, cid):
    liste_id_ = liste_id_par_cid[cid]
    valeurs_struct_par_id_ = valeurs_struct_par_id_par_cid[cid] 
    valeurs_version_par_id_ = valeurs_version_par_id_par_cid[cid] 
    
    valeurs_cid = []
    for id_ in liste_id_:
        assert valeurs_struct_par_id_[id_]['ID'] == id_
        assert valeurs_version_par_id_[id_]['ID'] == id_
        assert valeurs_struct_par_id_[id_]['ID_ELI'] == valeurs_version_par_id_[id_]['ID_ELI']
        assert valeurs_struct_par_id_[id_]['ID_ELI_ALIAS'] == valeurs_version_par_id_[id_]['ID_ELI_ALIAS']
        assert valeurs_struct_par_id_[id_]['ANCIEN_ID'] == valeurs_version_par_id_[id_]['ANCIEN_ID']
        assert valeurs_struct_par_id_[id_]['ORIGINE'] == valeurs_version_par_id_[id_]['ORIGINE']
        assert valeurs_struct_par_id_[id_]['NATURE'] == valeurs_version_par_id_[id_]['NATURE']
        assert valeurs_struct_par_id_[id_]['CID'] == cid
        assert valeurs_version_par_id_[id_]['CID'] == cid
        assert valeurs_struct_par_id_[id_]['NUM'] == valeurs_version_par_id_[id_]['NUM']
        assert valeurs_struct_par_id_[id_]['NUM_SEQUENCE'] == valeurs_version_par_id_[id_]['NUM_SEQUENCE']
        assert valeurs_struct_par_id_[id_]['NOR'] == valeurs_version_par_id_[id_]['NOR']
        assert valeurs_struct_par_id_[id_]['DATE_PUBLI'] == valeurs_version_par_id_[id_]['DATE_PUBLI']
        assert valeurs_struct_par_id_[id_]['DATE_TEXTE'] == valeurs_version_par_id_[id_]['DATE_TEXTE']
        assert valeurs_struct_par_id_[id_]['DERNIERE_MODIFICATION'] == valeurs_version_par_id_[id_]['DERNIERE_MODIFICATION']
        assert valeurs_struct_par_id_[id_]['VERSIONS_A_VENIR'] == valeurs_version_par_id_[id_]['VERSIONS_A_VENIR']
        assert valeurs_struct_par_id_[id_]['ORIGINE_PUBLI'] == valeurs_version_par_id_[id_]['ORIGINE_PUBLI']
        assert valeurs_struct_par_id_[id_]['PAGE_DEB_PUBLI'] == valeurs_version_par_id_[id_]['PAGE_DEB_PUBLI']
        assert valeurs_struct_par_id_[id_]['PAGE_FIN_PUBLI'] == valeurs_version_par_id_[id_]['PAGE_FIN_PUBLI']
        
        valeurs_id_ = {
            'base_origine': base_origine,
            'categorie': categorie,
            'cid': cid,
            'id_': id_,
            'valide': True,
            
            'ID_ELI': valeurs_struct_par_id_[id_]['ID_ELI'],
            'ID_ELI_ALIAS': valeurs_struct_par_id_[id_]['ID_ELI_ALIAS'],
            'ANCIEN_ID': valeurs_struct_par_id_[id_]['ANCIEN_ID'],
            'ORIGINE': valeurs_struct_par_id_[id_]['ORIGINE'],
            'URL_version': valeurs_version_par_id_[id_]['URL'],
            'URL_struct': valeurs_struct_par_id_[id_]['URL'],
            'NATURE': valeurs_struct_par_id_[id_]['NATURE'],
            'NUM': valeurs_struct_par_id_[id_]['NUM'],
            'NUM_SEQUENCE': valeurs_struct_par_id_[id_]['NUM_SEQUENCE'],
            'NOR': valeurs_struct_par_id_[id_]['NOR'],
            'DATE_PUBLI': valeurs_struct_par_id_[id_]['DATE_PUBLI'],
            'DATE_TEXTE': valeurs_struct_par_id_[id_]['DATE_TEXTE'],
            'DERNIERE_MODIFICATION': valeurs_struct_par_id_[id_]['DERNIERE_MODIFICATION'],
            'ORIGINE_PUBLI': valeurs_struct_par_id_[id_]['ORIGINE_PUBLI'],
            'PAGE_DEB_PUBLI': valeurs_struct_par_id_[id_]['PAGE_DEB_PUBLI'],
            'PAGE_FIN_PUBLI': valeurs_struct_par_id_[id_]['PAGE_FIN_PUBLI'],

            'TITRE': valeurs_version_par_id_[id_]['TITRE'],
            'TITREFULL': valeurs_version_par_id_[id_]['TITREFULL'],
            'ETAT': valeurs_version_par_id_[id_]['ETAT'],
            'DATE_DEBUT': valeurs_version_par_id_[id_]['DATE_DEBUT'],
            'DATE_FIN': valeurs_version_par_id_[id_]['DATE_FIN'],
            'AUTORITE': valeurs_version_par_id_[id_]['AUTORITE'],
            'MINISTERE': valeurs_version_par_id_[id_]['MINISTERE'],
            'MCS_TXT': valeurs_version_par_id_[id_]['MCS_TXT'],
            'NOTICE': valeurs_version_par_id_[id_]['NOTICE'],
            'VISAS': valeurs_version_par_id_[id_]['VISAS'],
            'SIGNATAIRES': valeurs_version_par_id_[id_]['SIGNATAIRES'],
            'TP': valeurs_version_par_id_[id_]['TP'],
            'NOTA': valeurs_version_par_id_[id_]['NOTA'],
            'ABRO': valeurs_version_par_id_[id_]['ABRO'],
            'RECT': valeurs_version_par_id_[id_]['RECT'],
            'SM': valeurs_version_par_id_[id_]['SM'],
        }
        
        valeurs_id_['VERSIONS_A_VENIR'] = json.dumps(list(valeurs_struct_par_id_[id_]['VERSIONS_A_VENIR']))
        valeurs_id_['LIENS'] = json.dumps([dict(fd) for fd in valeurs_version_par_id_[id_]['LIENS']])
        valeurs_id_['VERSIONS'] = json.dumps([dict(fd) for fd in valeurs_struct_par_id_[id_]['VERSIONS']])
        valeurs_id_['LIENS_ART'] = json.dumps([dict(fd) for fd in valeurs_struct_par_id_[id_]['LIENS_ART']])
        valeurs_id_['LIENS_SECTION_TA'] = json.dumps([dict(fd) for fd in valeurs_struct_par_id_[id_]['LIENS_SECTION_TA']])
        
        valeurs_cid.append(valeurs_id_)
        
    return valeurs_cid


In [26]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [19]:
valeurs = []
for base_origine, categorie, cid in liste_texte:
    valeurs += traite_cid(base_origine, categorie, cid)

In [23]:
liste_champ = list(valeurs[0].keys())

In [27]:
liste_chaine = [
    curseur.mogrify("(" + ','.join(["%s"] * len(liste_champ)) + ")",
            [v[c] for c in liste_champ]
        )
    for v in valeurs
]


In [76]:
nb_fournee = 1000
for i_fournee in range(len(liste_chaine)//nb_fournee + 1):
    sous_liste = liste_chaine[i_fournee*nb_fournee:(i_fournee+1)*nb_fournee]
    chaine_valeurs = b','.join(sous_liste)
    
    curseur.execute(b"insert into texte_version (" + b','.join([bytearray(c, 'utf-8') for c in liste_champ]) + b") values " + chaine_valeurs + b";")
    connection.commit()

In [77]:
curseur.close()
connection.close()